In [4]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd
import numpy as np

BACKTESTING_MODULE_PATH = os.path.abspath('../../backtest')
sys.path.insert(1, BACKTESTING_MODULE_PATH)
from backtesting.backtesting import Backtest, Strategy

METHOD_MODULE_PATH = os.path.abspath('../..')
sys.path.insert(1, METHOD_MODULE_PATH)
import method.Ticker as Ticker
import method.SaleRules as _sr
import method.algofuncs as _af
path = os.getcwd()

In [5]:
class FollowTheTrend(Strategy):
    def init(self):
        self.buy_price = 0
        self._index = 0
    def next(self):
        prices = self.data.Close
        opens = self.data.Open
        volumes = self.data.Volume
        lows = pd.DataFrame(data=self.data.Low).Low
        highs = pd.DataFrame(data=self.data.High).High
        if len(prices) > 23:
            last_price = prices[-1]
            atr_22 = _sr.getATR(22, highs, lows)
            if self.buy_price == 0 and Ticker.isFollowTrendingV2(prices, volumes, opens[-1], 2.5):
                self.buy_price = last_price
                self.buy()
            if self.buy_price != 0 and ((_sr.takeProfitByPercent(5, last_price, self.buy_price) and opens[-1] < prices[-1] and abs(highs.iloc[-1] - lows.iloc[-1]) > atr_22) or _sr.shouldCutLossByPercent(8, last_price, self.buy_price)):
                self.position.close()
                self.buy_price = 0

In [8]:
DATA_PATH = os.path.abspath('../../vn-stock-data/VNX/')
vn30_ticker = _af.getListVN30()
reports = [['Ticker', 'Avg. Trade [%]', 'Equity Final [$]', 'Return [%]', 'Sharpe Ratio', 'Win Rate [%]', 'Trades']]
for ticker_id in vn30_ticker:
    ticker = _af.get_pricing_by_path(DATA_PATH + '/' + ticker_id+'.csv', '2010-01-01', '2020-10-05')
    bt = Backtest(ticker, FollowTheTrend, commission=.005, exclusive_orders=False)
    stats = bt.run()
    if stats['_trades'].shape[0]:
        reports = np.append(reports, [[ticker_id, stats['Avg. Trade [%]'], stats['Equity Final [$]'], stats['Return [%]'], stats['Sharpe Ratio'], stats['Win Rate [%]'], stats['_trades'].shape[0]]], axis=0)
ls = pd.DataFrame(data=reports[1:,1:],index=reports[1:,0],columns=reports[0,1:])
ls_s = ls.sort_values(by=['Avg. Trade [%]'], ascending=False, na_position='first')
ls_s


,Avg. Trade [%],Equity Final [$],Return [%],Sharpe Ratio,Win Rate [%],Trades
NVL,5.850484179788329,10583.853564000001,5.83853564000001,0.7139511313113412,100.0,1
PNJ,5.150213444017093,12216.588989000007,22.16588989000007,0.43908206209638473,100.0,4
MBB,2.52051828278943,20069.885272500047,100.69885272500048,0.6166321187087852,82.14285714285714,28
VNM,2.15292582278257,12628.251991500007,26.28251991500007,0.28731800058750384,72.72727272727273,11
REE,1.5195113701196528,16692.492079500047,66.92492079500047,0.37141298705685216,67.64705882352942,34
VCB,1.494807280887267,13449.526911000023,34.49526911000022,0.2661461385677359,75.0,20
VJC,1.324426761698927,10405.659135500002,4.056591355000019,0.12205686773700243,66.66666666666666,3
SSI,0.4738089451240768,11633.40462200005,16.33404622000049,0.1325485827135057,65.625,32
MWG,0.3769492339879221,10837.968693000013,8.379686930000133,0.12429392029534499,62.5,8
VIC,0.36691479459383114,10726.498059000018,7.264980590000178,0.06038122410175167,63.1578947368421,19
